In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.applications import EfficientNetV2M, ConvNeXtBase
import numpy as np
import gc
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    

2024-05-02 16:41:33.749301: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 16:41:34.669501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/tobias/miniconda3/env

Current GPU allocator: cuda_malloc_async
Setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-05-02 16:41:36.179001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 16:41:36.179069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
study_name = '429_convnextBase_003_998_1'

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']



In [3]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [4]:

pickle_file_path = './data/test_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = './data/train_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [5]:
import glob

directory_path = './NN_search'
pattern = f"{directory_path}/{study_name}*.h5"

files = glob.glob(pattern)

max_r2_score = float('-inf')
best_model = None

# Käy läpi jokainen tiedosto ja etsi suurin r2_score_inv
for file in files:
    value = float(file.split('best_val')[1].split('_')[1])
    if value > max_r2_score:
        max_r2_score = value
        best_model = file


# Tulosta suurin löydetty r2_score_inv ja vastaava tiedosto
print(f"Best R2-score: {max_r2_score:.5f}")
if best_model:
    print(f"Best model: {best_model}")
else:
    print("No best model found")

best_log_transforms_name =  f'./NN_search/{study_name}_{max_r2_score:.5f}_best_log_transforms.pickle'
best_scalers_name = f'./NN_search/{study_name}_{max_r2_score:.5f}_best_scalers.pickle'

print(f'Opening log transforms from {best_log_transforms_name}')
with open(best_log_transforms_name, 'rb') as f:
    log_transforms = pickle.load(f)

print(f'Opening scalers from {best_scalers_name}')
with open(best_scalers_name, 'rb') as f:
    scaler_transforms = pickle.load(f)




def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')
    
custom_objects = {"r2_score_tf": r2_score_tf}

nas_model  = tf.keras.models.load_model(best_model, custom_objects=custom_objects)



        

Best R2-score: 0.39703
Best model: ./NN_search/429_convnextBase_003_998_1_best_val_0.39703_model.h5
Opening log transforms from ./NN_search/429_convnextBase_003_998_1_0.39703_best_log_transforms.pickle
Opening scalers from ./NN_search/429_convnextBase_003_998_1_0.39703_best_scalers.pickle


2024-05-02 16:41:36.735661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 16:41:36.735766: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 16:41:36.735814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 16:41:36.927268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 16:41:36.927479: I tensorflow/compile

In [6]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [12]:

tf.keras.backend.clear_session()
gc.collect()

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()

features_test = np.stack(test_df['features_avg'].values)
# print(features_test)
print(type(features_test))

predictions = nas_model.predict(features_test, verbose=1)

predictions = scaler_transforms.inverse_transform(predictions)

for i, target in enumerate(mean_columns):
    print(f'Logpot transforming target: : {target}, log transform : {log_transforms[target]}')
    log_base = log_transforms[target] 
    predictions[:, i] = np.power(log_base, predictions[:, i])
 


<class 'numpy.ndarray'>
 11/205 [>.............................] - ETA: 3s

2024-05-02 16:43:36.336685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


205/205 [==============================] - 1s 3ms/step
Logpot transforming target: : X4_mean, log transform : 7
Logpot transforming target: : X11_mean, log transform : 4
Logpot transforming target: : X18_mean, log transform : 7
Logpot transforming target: : X50_mean, log transform : 9
Logpot transforming target: : X26_mean, log transform : 11
Logpot transforming target: : X3112_mean, log transform : 9


In [13]:
train_df[mean_columns].describe()

,X4_mean,X11_mean,X18_mean,X50_mean,X26_mean,X3112_mean
count,53860.000000,53860.000000,53860.000000,53860.000000,53860.000000,53860.000000
mean,0.523042,15.913825,3.341783,1.627172,53.981490,2007.993034
std,0.147139,8.053792,5.665402,0.685494,258.422294,3888.581538
min,0.052820,2.349170,0.026446,0.403863,0.003763,7.154126
25%,0.411820,10.724026,0.316243,1.180538,0.583473,262.296271
50%,0.509048,15.104080,0.717389,1.480481,2.557434,732.011634
75%,0.621865,19.583160,3.543715,1.916863,14.706685,2129.828428
max,1.113824,79.383585,38.536136,6.353120,4811.793346,50147.882195


In [14]:
predictions_df = pd.DataFrame(predictions, columns=mean_columns)
predictions_df.describe()

,X4_mean,X11_mean,X18_mean,X50_mean,X26_mean,X3112_mean
count,6545.000000,6545.000000,6545.000000,6545.000000,6545.000000,6545.000000
mean,0.514915,15.831338,2.634717,1.514076,22.283663,1312.517700
std,0.111881,5.622025,4.141629,0.435058,68.636917,1599.304932
min,0.243768,3.103024,0.050461,0.425071,0.005881,13.392311
25%,0.433630,12.781852,0.353344,1.227581,0.831090,317.931946
50%,0.497790,15.947679,0.601210,1.451434,2.375526,653.618286
75%,0.593817,18.638266,2.783961,1.689676,12.950400,1730.580444
max,0.854950,52.472149,32.611366,3.522880,1057.548340,20662.169922


In [15]:

target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission_df[target_columns] = predictions
print(submission_df.info())

submission_df.to_csv('./data/submission.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6545 entries, 0 to 6544
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      6545 non-null   int64  
 1   X4      6545 non-null   float32
 2   X11     6545 non-null   float32
 3   X18     6545 non-null   float32
 4   X50     6545 non-null   float32
 5   X26     6545 non-null   float32
 6   X3112   6545 non-null   float32
dtypes: float32(6), int64(1)
memory usage: 204.7 KB
None
